In [18]:
# scraping from https://www.bestfightodds.com/

#needed after restructuring of directory
import os
os.chdir('/Users/alexchandler/UFC_Prediction_2022')

#!/usr/bin/env python
# coding: utf-8

# In[ ]:

import requests
import urllib3
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime, timedelta
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

#turning csv files to json files
import csv
import json

 
# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath, column):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            # primary key given by column variable
            key = rows[column]
            data[key] = rows
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
        
def get_odds_two_rows_per_fight():
    url = 'https://www.bestfightodds.com'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser") 
    mydivs = soup.find_all("tr", {"class": ""})
    rows=[tr for tr in mydivs if 'bestbet' in str(tr)]
    names=[]
    oddsDicts=[]
    books=['DraftKings','BetMGM','Caesars','BetRivers','FanDuel','PointsBet','Unibet','Bet365','BetWay','5D','Ref']
    for row in rows:
        #gets name of fighter in row
        name = row.find_all("span", {"class": "t-b-fcc"})[0].text
        oddsList=[name]
        i=0
        for stat in row.select('td'):
            i+=1
            if i>11:
                break
            try:
                odds = stat.select('span')[0].text
                oddsList.append(odds)
            except:
                oddsList.append('')
        names.append(name)
        oddsDicts.append(dict(zip(['name']+books,oddsList)))
    oddsDict = dict(zip(names,oddsDicts))
    names = list(oddsDict.keys())
    row0=oddsDict[list(oddsDict.keys())[0]]
    odds_df = pd.DataFrame(row0, index=[0])
    for i in range(1,len(names)):
        row=oddsDict[names[i]]
        odds_df = pd.concat([odds_df, pd.DataFrame(row, index=[i])], axis = 0)
    return odds_df


#problem: the fights in the "future events" category do not get lined up properly
def get_odds():
    url = 'https://www.bestfightodds.com'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser") 
    mydivs = soup.find_all("tr", {"class": ""})
    rows=[tr for tr in mydivs if 'bestbet' in str(tr)]
    names=[]
    oddsDicts=[]
    books=['DraftKings','BetMGM','Caesars','BetRivers','FanDuel','PointsBet','Unibet','Bet365','BetWay','5D','Ref']
    for row in rows:
        #gets name of fighter in row
        name = row.find_all("span", {"class": "t-b-fcc"})[0].text
        oddsList=[name]
        i=0
        for stat in row.select('td'):
            i+=1
            if i>11:
                break
            try:
                odds = stat.select('span')[0].text
                oddsList.append(odds)
            except:
                oddsList.append('')
        names.append(name)
        oddsDicts.append(dict(zip(['name']+books,oddsList)))
    oddsDict = dict(zip(names,oddsDicts))
    names = list(oddsDict.keys())
    row0=oddsDict[list(oddsDict.keys())[0]]
    odds_df = pd.DataFrame(row0, index=[0])
    for i in range(1,len(names)):
        row=oddsDict[names[i]]
        odds_df = pd.concat([odds_df, pd.DataFrame(row, index=[i])], axis = 0)
    #making it so each fight has just a single row instead of two rows
    #making dataframe just for even indexed columns
    odds_df_evens = odds_df[odds_df.index%2==0]
    newcolumns={}
    for col in list(odds_df_evens.columns):
        newcolumns[col]='fighter '+col
    odds_df_evens=odds_df_evens.rename(columns=newcolumns)
    odds_df_evens.reset_index(drop=True, inplace=True)
    #making dataframe just for odd indexed columns
    odds_df_odds = odds_df[odds_df.index%2==1]
    newcolumns={}
    for col in list(odds_df_odds.columns):
        newcolumns[col]='opponent '+col
    odds_df_odds=odds_df_odds.rename(columns=newcolumns)
    odds_df_odds.reset_index(drop=True, inplace=True)
    new_odds_df = pd.concat([odds_df_evens, odds_df_odds], axis = 1);new_odds_df
    
    return new_odds_df


In [19]:
odds_df = get_odds();odds_df

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref
0,Amanda Lemos,+190,+190,+190,+165,+176,+180,+165,+175,+175,+195,+185,Jessica Andrade,-235,-225,-230,-210,-210,-220,-210,-213,-225,-215,-220
1,Claudio Puelles,-110,-110,-110,-113,-106,-110,-113,-111,-110,+100,-115,Clay Guida,-110,-110,-110,-113,-110,-110,-113,-111,-110,-110,-105
2,Maycee Barber,-180,-190,-175,-195,-190,-190,-195,-175,-188,-190,-180,Montana de La Rosa,+155,+155,+150,+150,+160,+155,+150,+150,+150,+175,+155
3,Alexandr Romanov,-1500,-2500,-1500,-1430,-1500,-2000,-1430,-1429,-2000,-1675,-2500,Chase Sherman,+850,+1050,+850,+700,+870,+975,+700,+800,+900,+1150,+1200
4,Charles Jourdain,-120,-110,-120,-122,-120,-120,-122,-125,-120,-113,-117,Lando Vannata,+100,-110,+100,-103,+102,+100,-103,+100,-105,+103,-103
5,Jordan Wright,+165,+165,+165,+150,+160,+160,+150,+162,+163,+170,+169,Marc-Andre Barriault,-195,-200,-195,-186,-190,-195,-186,-189,-200,-185,-194
6,Dwight Grant,-105,+105,+100,+101,+100,+100,+101,+100,-105,+102,-102,Sergey Khandozhko,-115,-125,-120,-125,-118,-120,-125,-125,-120,-112,-118
7,Isaac Villanueva,+450,+475,+450,+410,+460,+425,+410,+450,+400,+480,+425,Tyson Pedro,-630,-700,-600,-625,-620,-600,-625,-625,-549,-570,-650
8,Aori Qileng,-220,,,,-240,,,,-225,-240,-225,Cameron Else,,+200,+190,+190,,+195,+190,+200,,,
9,Qileng Aori,,-250,-230,-250,,-240,-250,-250,,,,Evan Elder,+110,+110,+110,+108,+114,+112,+108,+100,+110,+115,+111


In [20]:
result = odds_df.to_json()
parsed = json.loads(result)
jsonFilePath='src/models/buildingMLModel/data/external/vegas_odds.json'
with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(parsed, indent=4))
print('saved to '+jsonFilePath)

saved to src/models/buildingMLModel/data/external/vegas_odds.json


In [ ]:
odds_df_two=get_odds_two_rows_per_fight();odds_df_two